In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

#import numpy as np # linear algebra
#import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
       # print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#2Approach to detect datasources: Countvectorizer with n-grams and 
import re,nltk,gensim
import json as js
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from gensim.models.ldamodel import LdaModel

def get_data_sources(tex):
    f_hand=open(tex,'r')
    text=js.load(f_hand)
    #convert json to string
    full_text=''
    for i in range(0,len(text)):
        full_text=full_text+' '+text[i]['text']
    #free from punctuation pharantheses and dates    
    # FIND CAPITAL LETTERS
    capital_letters=re.findall(r'([A-Z]{3,})',full_text)
    capitalletters=list(capital_letters)
    dict_capitalletters={}
    for i in capitalletters:
        dict_capitalletters[i]=dict_capitalletters.get(i,0)+1
    #dict_capitalletters
    #REPLACE CAPICAL LETTERS with long versions
    #here we had to remove stop words in order to find out long versions of datasources in text
    ffs=[w for w in nltk.word_tokenize(full_text) if w.lower() not in stopwords.words('english')]
    ffs_text=' '.join(ffs)
    for i in ffs:
        for cap_let in set(capitalletters):
            if i==cap_let:
                word_to_find=[cl+'[A-Za-z]+' for cl in cap_let]
                wrdtofind=' '.join(word_to_find)
                #print(wrdtofind)
                #print(re.search(wrdtofind,ffs_text))
                try:
                    ffs_text=re.sub(i,re.search(wrdtofind ,ffs_text)[0],ffs_text)
                    #ffs_text=re.sub(i,re.search(wrdtofind.lower(),ffs_text)[0],ffs_text)
                except:
                    continue
            else:
                continue

    #free from punctuation and pharantheses
    free_from_symb_dates=[]
    for i in nltk.sent_tokenize(ffs_text):
        free_from_symb_dates.append(re.sub(r'\b\w\w\b','',re.sub(r'\d+','',re.sub(r'\W+',' ', i)))+'.')

    #data Sentences 
    data_words=['Study','Survey','Statistics','Data','Dataset','Database','Census','Assessment']

    data_sentences=[w for w in free_from_symb_dates for i in range(0,len(data_words)) if data_words[i].lower() in w.lower()]
    #countvectorizer and fit and transform text 
    vectorizer=CountVectorizer(analyzer='word',max_df=0.7,ngram_range=(3,5))# ngram_range=(3,5) ,
    fitted_model=vectorizer.fit_transform(set(data_sentences))
    # create corpus to use in Lda model
    corpus = gensim.matutils.Sparse2Corpus(fitted_model, documents_columns=False)
    id_map = dict((v, k) for k, v in vectorizer.vocabulary_.items())

    # Lda model
    ldamodel=LdaModel(corpus,num_topics=1,id2word=id_map,per_word_topics=True,passes=50)
    return ldamodel.show_topic(topicid=0,topn=4)[0][0]+'|'+ldamodel.show_topic(topicid=0,topn=4)[1][0]+'|'+ldamodel.show_topic(topicid=0,topn=4)[2][0]

header='Id,PredictionString'
file_open=open('submission.csv','w')
file_open.write(header+"\n")
file_open.close()
text_ids=['2100032a-7c33-4bff-97ef-690822c43466',
           '2f392438-e215-4169-bebf-21ac4ff253e1',
           '3f316b38-1a24-45a9-8d8c-4e05a42257c6',
           '8e6996b4-ca08-4c0b-bed2-aaf07a4c6a60']
    
with open('submission.csv', 'a') as f:
    for ids in text_ids:
        tex='../input/coleridgeinitiative-show-us-the-data/test/'+ids+'.json'
        f.write(ids+ ','+get_data_sources(tex)+"\n")
f.close()